<a href="https://colab.research.google.com/github/sujinee01/Capstone/blob/main/TCNmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import pytorch_lightning as pl
except ModuleNotFoundError:
    !pip install pytorch-lightning
#라이브러리 import
import pandas as pd
import numpy as np
import torch1
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset,TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

import pytorch_lightning as pl
from torch.utils.data import DataLoader
drive.mount('/content/drive')


# 데이터 로드 및 전처리
data = pd.read_excel("/content/drive/My Drive/LiFePO4 Dynamic Profile Files/LiFePO4 Dynamic Profile Files/-10도/LiFePO4_DST_SOC_N10_1.xlsx")

# 데이터 선택 (스케일링 전)
data_selected = data.loc[:, ["Current(A)", "Voltage(V)", "Temperature (C)_1"]].copy()  # 헤더 포함한 전체 데이터 선택
soc_data = data.loc[:,["SOC(t)"]].copy()  # 헤더 포함한 전체 데이터 선택

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_selected)

data_scaled = pd.DataFrame(data_scaled, columns=["Current(A)", "Voltage(V)", "Temperature (C)_1"])


# 테스트 데이터 비율 설정
test_size = 0.2

# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data_scaled, soc_data.values, test_size=0.2, random_state=42)

#TCN 모델
class TCN(pl.LightningModule):
    def __init__(self, input_size, output_size, num_channels, kernel_size):
        super().__init__()

        if isinstance(input_size, tuple):
            in_channels = input_size[0]
        else:
            in_channels = input_size

        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels=in_channels, out_channels=num_channels, kernel_size=kernel_size, padding="same"),
            nn.ReLU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, kernel_size=kernel_size, padding="same"),
            nn.ReLU(),
        ])

        self.linear_layer = nn.Linear(in_features=num_channels, out_features=output_size)

        # 옵티마이저 정의
        self.optimizer = torch.optim.Adam(self.parameters())

    def forward(self, x):
        for i, conv_layer in enumerate(self.conv_layers):
            x = conv_layer(x)
            if i % 2 == 1:  # ReLU layer
                x = x.detach()

        x = x.mean(dim=2)

        x = self.linear_layer(x)

        return x


    # 학습 단계 정의 (수정)
    def training_step(self, batch, batch_idx):
        # 데이터 로더에서 받은 데이터 분리
        x, y = batch

        # 모델 예측
        logits = self(x).squeeze()

        # 손실 계산 (이진 분류 문제이므로 BCE 손실 함수 사용)
        loss = nn.BCEWithLogitsLoss()(logits, y)

        # 로그 기록
        self.log("train_loss", loss)

        # 최적화 알고리즘 업데이트
        self.optimizer.zero_grad()
        loss.backward(retain_graph=True)  # 그래프 보존 설정
        self.optimizer.step()

        return loss


    # 최적화 알고리즘 설정 (수정)
    def configure_optimizers(self):
        # 옵티마이저 반환
        return self.optimizer



# 모델 하이퍼파라미터 설정
input_size = 3
output_size = 1
num_channels = 16
kernel_size = 3

# 모델 생성
model = TCN(input_size, output_size, num_channels, kernel_size)

# 학습률 및 에포크 수 설정
lr = 0.001
epochs = 100

# Trainer 정의
trainer = pl.Trainer(
    default_root_dir="/content/drive/My Drive/LiFePO4 Dynamic Profile Files",
    max_epochs=epochs,
    accelerator="auto"
)



# 데이터 로더 생성
train_dataset = [(torch.tensor(x).unsqueeze(1).float(), torch.tensor(y).float().squeeze()) for x, y in zip(X_train.values, y_train)]
test_dataset = [(torch.tensor(x).unsqueeze(1).float(), torch.tensor(y).float().squeeze()) for x, y in zip(X_test.values, y_test)]

train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)



# 모델 학습

trainer.fit(model, train_dataloader)



# 모델의 예측값 계산
model.eval()  # 모델을 평가 모드로 설정
predictions = []
with torch.no_grad():  # 그래디언트 계산 비활성화
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        predictions.extend(outputs.squeeze().tolist())

# 예측값과 실제값의 MSE 및 RMSE 계산
mse = nn.MSELoss()(torch.tensor(predictions), torch.tensor(y_test))
rmse = torch.sqrt(mse)

print("MSE:", mse.item())
print("RMSE:", rmse.item())



  Using cached pytorch_lightning-2.2.1-py3-none-any.whl (801 kB)
  Using cached torchmetrics-1.3.2-py3-none-any.whl (841 kB)
  Using cached lightning_utilities-0.11.2-py3-none-any.whl (26 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 124.2/124.2 MB 70.4 MB/s eta 0:00:01
ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'torch1'